# Feature Engineering and Model Training for Per-Query Optimization

This notebook goes through the process of feature engineering based on the query set used for the global hybrid search optimization notebook.

It tries out different feature combinations and model types to identify a useful way forward towards optimizing queries dynamically.

Notes:

* `searcharray` produced an error with numpy version 2.1.2
* Make sure to use `numpy==1.26.4`

Error fixed in the meantime (not verified)

In [175]:
import pandas as pd
import math
import string
import requests
import json
from searcharray import SearchArray
import numpy as np
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from tqdm.notebook import tqdm_notebook

In [192]:
df_metrics_train = pd.read_csv('../data/metrics_query_train_small.csv')
df_metrics_test = pd.read_csv('../data/metrics_query_test_small.csv')
df_metrics = pd.concat([df_metrics_train, df_metrics_test])
df_metrics.reset_index(inplace=True, drop=True)

In [193]:
df_metrics[df_metrics['metric'] == 'ndcg'].head(3)

,query,pipeline,metric,value
13200,09 g6 wheel not cover grey,l2arithmetic_mean0.0,ndcg,0.0
13201,09 g6 wheel not cover grey,l2arithmetic_mean0.1,ndcg,0.0
13202,09 g6 wheel not cover grey,l2arithmetic_mean0.2,ndcg,0.0


In [195]:
indexes = df_metrics[df_metrics['metric'] == 'ndcg'].groupby(by='query')['value'].idxmax()
df_filtered = df_metrics.loc[indexes, ['query','pipeline', 'value']]
df_filtered.reset_index(inplace=True, drop=True)

In [198]:
df_filtered.sort_values(by='query').head(5)

,query,pipeline,value
0,09 g6 wheel not cover grey,l2arithmetic_mean0.0,0.000000
1,1 1/4 pop up bathroom sink drain without overflow,l2arithmetic_mean0.4,0.044023
2,1 ‘ velcro without adhesive for sewing,l2arithmetic_mean0.4,0.086189
3,1.5 inch heel,l2arithmetic_mean0.4,0.062158
4,1/2 inch binder,l2geometric_mean0.1,0.158702


In [197]:
df_filtered.shape[0]

250

## Get 'neuralness' of a Query

`neuralness` is the weight of the neural query extracted by the pipeline name that was used to generate the NDCG value.

In [199]:
def get_neuralness(pipeline):
    keywordness = pipeline[-3:]

    try:
        neuralness = 1 - float(keywordness)
        return neuralness
    except ValueError:
        print("ERROR")
        return math.nan

In [200]:
df_filtered['neuralness'] = df_filtered.pipeline.apply(get_neuralness)

In [201]:
df_filtered = df_filtered.drop(columns=['pipeline'])

In [202]:
df_filtered.columns = ['query_string', 'ndcg', 'neuralness']

In [203]:
df_filtered.head(3)

,query_string,ndcg,neuralness
0,09 g6 wheel not cover grey,0.000000,1.0
1,1 1/4 pop up bathroom sink drain without overflow,0.044023,0.6
2,1 ‘ velcro without adhesive for sewing,0.086189,0.6


## Query Feature Functions

`num_of_terms`: we expect the number of query terms to be a good feature because the more query terms used the more specific a query is.

`query_length`: we expect the query length (number of characters) to be a good feature because the longer a query is the more specific it is.

`has_numbers`: we expect the presence of numbers/digits to be a good feature because numbers may point towards specific queries.

`has_special_char`: we expect the presence of special characters (non-alphanumeric characters) to be a good feature because that may point towards specific queries. 

In [204]:
def num_of_terms(query_string):
    terms = query_string.split(" ")
    return len(terms)

def query_length(query_string):
    return len(query_string)

def has_numbers(query_string):
    return int(any(char.isdigit() for char in query_string))

def has_special_char(query_string):
    # Define special characters (all non-alphanumeric characters)
    special_chars = string.punctuation
    # Return True if any character in the string is a special character
    return int(any(char in special_chars for char in query_string))

In [205]:
df_filtered['num_of_terms'] = df_filtered.query_string.apply(num_of_terms)
df_filtered['query_length'] = df_filtered.query_string.apply(query_length)
df_filtered['has_numbers'] = df_filtered.query_string.apply(has_numbers)
df_filtered['has_special_char'] = df_filtered.query_string.apply(has_special_char)

In [206]:
df_filtered.head(5)

,query_string,ndcg,neuralness,num_of_terms,query_length,has_numbers,has_special_char
0,09 g6 wheel not cover grey,0.000000,1.0,6,26,1,0
1,1 1/4 pop up bathroom sink drain without overflow,0.044023,0.6,9,49,1,1
2,1 ‘ velcro without adhesive for sewing,0.086189,0.6,7,38,1,0
3,1.5 inch heel,0.062158,0.6,3,13,1,1
4,1/2 inch binder,0.158702,0.9,3,15,1,1


## Result Feature Functions

`num_results_kw_search`: the number of results for the keyword query.

`max_doc_score`: max score of the retrieved top 10 titles. The scores are BM25 scores calculated individually per result set. That means that the document frequencies of terms is not calculated on the whole index but only on the retrieved subset for the query, making the scores more comparable to each other.

`sum_of_doc_scores`: we use the sum of the scores (and no average value) as an aggregate to have a measure of how relevant all retrieved top 10 titles are.

`get_semantic_scores_for_query`: retrieves the semantic scores for the neural query for the top 10. We store the maximum and the mean as result features.

### Helper functions

`get_titles_for_query`: retrieves the titles of the keyword matches for later score calculation

`whitespace_to_lower`: tokenizes and lowercases text

`get_model_id`: to run a neural search to get the semantic scores we need to know the id of the available model in OpenSearch


### Get Number of Results for Keyword Search

In [207]:
def num_results_kw_search(query_string):
    url = "http://localhost:9200/ecommerce/_search"

    headers = {
        'Content-Type': 'application/json'
    }
    
    
    
    payload = {
      "_source": {
        "excludes": [
          "title_embedding"
        ]
      },
      "query": {
        "multi_match" : {
          "type":       "best_fields",
          "fields":     [
            "product_id^100",
            "product_bullet_point^3",
            "product_color^2",
            "product_brand^5",
            "product_description",
            "product_title^10"
          ],
          "operator":   "and",
          "query":      query_string
        }
      },
      "track_total_hits": "true"
    }
    response = requests.request("POST", url, headers=headers, data=json.dumps(payload)).json()
    num_results = response['hits']['total']['value']
    return num_results

In [208]:
def get_titles_for_query(query_string):
    url = "http://localhost:9200/ecommerce/_search"

    headers = {
        'Content-Type': 'application/json'
    }
      
    payload = {
      "_source": {
        "excludes": [
          "title_embedding"
        ],
        "includes": "product_title"
      },
      "query": {
        "multi_match" : {
          "type":       "best_fields",
          "fields":     [
            "product_id^100",
            "product_bullet_point^3",
            "product_color^2",
            "product_brand^5",
            "product_description",
            "product_title^10"
          ],
          "operator":   "and",
          "query":      query_string
        }
      },
      "track_total_hits": "true"
    }
    response = requests.request("POST", url, headers=headers, data=json.dumps(payload)).json()
    titles = []
    num_results = response['hits']['total']['value']
    if num_results > 0:
        # get all titles
        titles = [item['_source']['product_title'] for item in response['hits']['hits']]

    df = pd.DataFrame(titles, columns=['title'])

    return df

In [209]:
tqdm_notebook.pandas()
df_filtered['num_results'] = df_filtered.query_string.progress_apply(num_results_kw_search)

  0%|          | 0/250 [00:00<?, ?it/s]

### Tokenizer Definition

We define a simple tokenizer that splits on whitespaces and lowercases.

In [210]:
def whitespace_to_lower(text):
    split = text.lower().split()
    return [token
        for token in split]

ToDo: make the following functions more efficient.

The following functions contain redundant code and only differ in the return value.

In [211]:
# we use searcharray to calculate the highest title score among the retrieved docs
def max_doc_score(df, query):
    # if we retrieved zero docs we return 0 as the max score
    if df.shape[0] == 0:
        return 0
    else:
        # index the titles with the defined tokenizer
        df['title_indexed'] = SearchArray.index(df['title'], tokenizer=whitespace_to_lower)
        # tokenize the query with the same tokenizer
        tokenized_query = df['title_indexed'].array.tokenizer(query)
        # calculate the score for each query term
        # the result is a matrix with number of titles x number of query term dimensions
        scores = np.asarray([df['title_indexed'].array.score(query_term)
                                    for query_term in tokenized_query])
        # calculate the sum of all query terms per title
        sums = [sum(group) for group in zip(*scores)]
        # return the maximum
        return max(sums)

# we use searcharray to calculate the sum of all title scores among the retrieved docs
def sum_of_doc_scores(df, query):
    # if we retrieved zero docs we return 0 as the max score
    if df.shape[0] == 0:
        return 0
    else:
        # index the titles with the defined tokenizer
        df['title_indexed'] = SearchArray.index(df['title'], tokenizer=whitespace_to_lower)
        # tokenize the query with the same tokenizer
        tokenized_query = df['title_indexed'].array.tokenizer(query)
        # calculate the score for each query term
        # the result is a matrix with number of titles x number of query term dimensions
        scores = np.asarray([df['title_indexed'].array.score(query_term)
                                    for query_term in tokenized_query])
        # calculate the sum of all query terms per title
        sums = [sum(group) for group in zip(*scores)]
        # return the sum of all title scores
        return sum(sums)

### Get Semantic Scores

First we get the model id, then we use the model id in the query to retrieve the semantic scores.

In [212]:
# Get model_id
# We are assuming that the installation has only one model. Change this if you have more models 
# and need to pick a specific one

def get_model_id():
    url = "http://localhost:9200/_plugins/_ml/models/_search"
    
    headers = {
        'Content-Type': 'application/json'
    }
    
    payload = {
      "query": {
        "match_all": {}
      },
      "size": 1
    }
    
    response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

    return response.json()['hits']['hits'][0]['_source']['model_id']

model_id = get_model_id()

In [213]:
def get_semantic_scores_for_query(query_string):
    url = "http://localhost:9200/ecommerce/_search"

    headers = {
        'Content-Type': 'application/json'
    }
    
    
    
    payload = {
      "_source": {
        "excludes": [
          "title_embedding"
        ],
        "includes": "product_title"
      },
      "query": {
        "neural": {
          "title_embedding": {
            "query_text": query_string,
            "k": 100,
            "model_id": model_id
          }
        }
      },
      "size": 10
    }
    response = requests.request("POST", url, headers=headers, data=json.dumps(payload)).json()
    titles = []
    num_results = response['hits']['total']['value']
    if num_results > 0:
        # get all titles
        scores = [item['_score'] for item in response['hits']['hits']]

    return scores

In [214]:
import logging, sys
logging.disable(sys.maxsize)

In [215]:
tqdm_notebook.pandas()
df_filtered['max_title_score'] = df_filtered['query_string'].progress_apply(lambda q: max_doc_score(get_titles_for_query(q), q))
df_filtered['sum_title_scores'] = df_filtered['query_string'].progress_apply(lambda q: sum_of_doc_scores(get_titles_for_query(q), q))

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

In [216]:
tqdm_notebook.pandas()
df_filtered['max_semantic_score'] = df_filtered['query_string'].progress_apply(lambda q: max(get_semantic_scores_for_query(q)))
df_filtered['avg_semantic_score'] = df_filtered['query_string'].progress_apply(lambda q: np.mean(get_semantic_scores_for_query(q)))

  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

In [217]:
df_filtered.head(5)

,query_string,ndcg,neuralness,num_of_terms,query_length,has_numbers,has_special_char,num_results,max_title_score,sum_title_scores,max_semantic_score,avg_semantic_score
0,09 g6 wheel not cover grey,0.000000,1.0,6,26,1,0,0,0.000000,0.000000,0.542725,0.529522
1,1 1/4 pop up bathroom sink drain without overflow,0.044023,0.6,9,49,1,1,91,0.329475,2.666837,0.746640,0.681141
2,1 ‘ velcro without adhesive for sewing,0.086189,0.6,7,38,1,0,0,0.000000,0.000000,0.643718,0.586100
3,1.5 inch heel,0.062158,0.6,3,13,1,1,78,1.030156,4.811234,0.625160,0.568985
4,1/2 inch binder,0.158702,0.9,3,15,1,1,294,0.549694,2.752247,0.709633,0.677592


In [218]:
df_filtered.columns

Index(['query_string', 'ndcg', 'neuralness', 'num_of_terms', 'query_length',
       'has_numbers', 'has_special_char', 'num_results', 'max_title_score',
       'sum_title_scores', 'max_semantic_score', 'avg_semantic_score'],
      dtype='object')

In [219]:
# rename the columns
df_filtered = df_filtered.rename(columns={"num_of_terms": "f_1_num_of_terms",
                            "query_length": "f_2_query_length",
                            "has_numbers": "f_3_has_numbers",
                            "has_special_char": "f_4_has_special_char",
                            "num_results": "f_5_num_results",
                            "max_title_score": "f_6_max_title_score",
                            "sum_title_scores": "f_7_sum_title_scores",
                            "max_semantic_score": "f_8_max_semantic_score",
                            "avg_semantic_score": "f_9_avg_semantic_score"
                           })

In [220]:
df_filtered.head(5)

,query_string,ndcg,neuralness,f_1_num_of_terms,f_2_query_length,f_3_has_numbers,f_4_has_special_char,f_5_num_results,f_6_max_title_score,f_7_sum_title_scores,f_8_max_semantic_score,f_9_avg_semantic_score
0,09 g6 wheel not cover grey,0.000000,1.0,6,26,1,0,0,0.000000,0.000000,0.542725,0.529522
1,1 1/4 pop up bathroom sink drain without overflow,0.044023,0.6,9,49,1,1,91,0.329475,2.666837,0.746640,0.681141
2,1 ‘ velcro without adhesive for sewing,0.086189,0.6,7,38,1,0,0,0.000000,0.000000,0.643718,0.586100
3,1.5 inch heel,0.062158,0.6,3,13,1,1,78,1.030156,4.811234,0.625160,0.568985
4,1/2 inch binder,0.158702,0.9,3,15,1,1,294,0.549694,2.752247,0.709633,0.677592


In [221]:
# Uncomment the thing you want to do

# write features if they're initially computed
# df_filtered.to_csv('../data/features_5000_queries.csv', index=False)
# get features if they're already present and you run the training process from here
# df_filtered = to_csv('../data/features_5000_queries.csv')

# Explore Different Model Training Options

We try out linear regression and random forest regression as two types of models.

We want to find out which feature combination is best by trying out all different combinations.

We apply cross validation for a statistically sound process.

We explore the effect of adding regularization to reduce the change of overfitting.

## Linear Regression with Feature Combinations and Cross Validation

In [222]:
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import make_scorer, root_mean_squared_error, r2_score
from itertools import combinations
import numpy as np
import pandas as pd

# Prepare features and target
feature_columns = [ "f_1_num_of_terms", "f_2_query_length", "f_3_has_numbers", "f_4_has_special_char", "f_5_num_results",
                   "f_6_max_title_score", "f_7_sum_title_scores", "f_8_max_semantic_score",  "f_9_avg_semantic_score"]
kws_features = ["f_5_num_results", "f_6_max_title_score", "f_7_sum_title_scores"]
neural_search_features = ["f_8_max_semantic_score",  "f_9_avg_semantic_score"]
query_features = ["f_1_num_of_terms",  "f_2_query_length", "f_3_has_numbers", "f_4_has_special_char"]
y = df_filtered['neuralness']  # Target column

# Initialize cross-validation strategy
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

# Define the model
regr = LinearRegression()

rmse_scorer = make_scorer(root_mean_squared_error, greater_is_better=False)

# Initialize variables to track the best combination
best_score = float('inf')
best_combination = None
best_metrics = {}
# We initialize df_metrics only for this run and append the results of the following runs
df_model = []
# Initialize df_model_metrics to collect metrics for this iteration of model training
df_model_metrics = []

# Try all feature combinations of length 2 to 9
for r in tqdm_notebook(range(2, len(feature_columns) + 1)):
    for combo in combinations(feature_columns, r):
        X = df_filtered[list(combo)]  # Select current feature subset
        
        # Perform cross-validation
        rmse_scores = cross_val_score(regr, X, y, cv=cv, scoring=rmse_scorer)
        
        # Calculate the mean metrics for the current combination
        mean_rmse = -np.mean(rmse_scores)  # Since RMSE scorer is negative for minimization

        # does the current combination contain keyword search features
        keyword_b = any(feature in combo for feature in kws_features)
        # does the current combination contain neural search features
        neural_b = any(feature in combo for feature in neural_search_features)
        # does the current combination contain only query features (no keyword or neural search features)
        query_b = any(feature in combo for feature in query_features)
        
        df_model_metrics.append(pd.DataFrame({
            "features": [combo]*5,
            "rmse_scores": rmse_scores,
            "keyword_f": [keyword_b]*5,
            "neural_f": [neural_b]*5,
            "query_f": [query_b]*5,
            "model": "lr",
            "regularization": False,
            }))
        
        # Update if the current combination is better
        if mean_rmse < best_score:
            best_score = mean_rmse
            best_combination = combo
            best_metrics = {
                'RMSE (mean)': mean_rmse
            }

df_metrics = pd.concat(df_model_metrics)

# Output the best feature combination and its metrics
print("Best Feature Combination:", best_combination)
print("Best Metrics:")
print(f"Cross-Validation RMSE (mean): {best_metrics['RMSE (mean)']}")

  0%|          | 0/8 [00:00<?, ?it/s]

Best Feature Combination: ('f_1_num_of_terms', 'f_5_num_results', 'f_7_sum_title_scores', 'f_9_avg_semantic_score')
Best Metrics:
Cross-Validation RMSE (mean): 0.2633574367986088


## Linear Regression with Feature Combinations, Cross Validation and L2 Regularization

### Find best regularization parameter

Use all features, get the "best" regularization parameter with cross-validation by trying out a range

In [223]:
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.metrics import make_scorer, root_mean_squared_error, r2_score
from itertools import combinations
import numpy as np
import pandas as pd

# Prepare features and target
feature_columns = [ "f_1_num_of_terms", "f_2_query_length", "f_3_has_numbers", "f_4_has_special_char", "f_5_num_results",
                   "f_6_max_title_score", "f_7_sum_title_scores", "f_8_max_semantic_score",  "f_9_avg_semantic_score"]
kws_features = ["f_5_num_results", "f_6_max_title_score", "f_7_sum_title_scores"]
neural_search_features = ["f_8_max_semantic_score",  "f_9_avg_semantic_score"]
query_features = ["f_1_num_of_terms",  "f_2_query_length", "f_3_has_numbers", "f_4_has_special_char"]
y = df_filtered['neuralness']  # Target column

# Initialize cross-validation strategy
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

# Define alpha values to test in regularization
alpha_vals = [0.001, 0.1, 0.2, 0.5, 1.0, 1.5, 2.0, 5.0, 10.0]

# Initialize variables to track the best combination
best_metrics = {}
best_score = float('inf')
for alpha in alpha_vals:
    
    # Define the model
    regr = Ridge(alpha=alpha)

    rmse_scorer = make_scorer(root_mean_squared_error, greater_is_better=False)

    X = df_filtered[feature_columns]  # Select current feature subset
    
    # Perform cross-validation
    rmse_scores = cross_val_score(regr, X, y, cv=cv, scoring=rmse_scorer)
    
    mean_rmse = -np.mean(rmse_scores)  # Since RMSE scorer is negative for minimization
        
    # Update if the current combination is better
    if mean_rmse < best_score:
        print(f"Found lower mean RMSE ({mean_rmse}) for alpha value of {alpha}")
        best_score = mean_rmse
        best_metrics = {
            'alpha': alpha,
            'RMSE (mean)': best_score
        }

# Output the best feature combination and its metrics
print("Best Metrics:")
#print(f"Accuracy (mean): {best_metrics['Accuracy (mean)']}")
print(f"alpha: {best_metrics['alpha']}")
print(f"Cross-Validation RMSE (mean): {best_metrics['RMSE (mean)']}")

Found lower mean RMSE (0.27003352982087214) for alpha value of 0.001
Found lower mean RMSE (0.26952092870350997) for alpha value of 0.1
Found lower mean RMSE (0.26934471186149855) for alpha value of 0.2
Found lower mean RMSE (0.2691170261524326) for alpha value of 0.5
Found lower mean RMSE (0.2688805828423716) for alpha value of 1.0
Found lower mean RMSE (0.2686798290018556) for alpha value of 1.5
Found lower mean RMSE (0.26849882472534536) for alpha value of 2.0
Found lower mean RMSE (0.26767963067932565) for alpha value of 5.0
Found lower mean RMSE (0.26687696316635584) for alpha value of 10.0
Best Metrics:
alpha: 10.0
Cross-Validation RMSE (mean): 0.26687696316635584


In [224]:
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.metrics import make_scorer, root_mean_squared_error, r2_score
from itertools import combinations
import numpy as np
import pandas as pd

# Prepare features and target
feature_columns = [ "f_1_num_of_terms", "f_2_query_length", "f_3_has_numbers", "f_4_has_special_char", "f_5_num_results",
                   "f_6_max_title_score", "f_7_sum_title_scores", "f_8_max_semantic_score",  "f_9_avg_semantic_score"]
kws_features = ["f_5_num_results", "f_6_max_title_score", "f_7_sum_title_scores"]
neural_search_features = ["f_8_max_semantic_score",  "f_9_avg_semantic_score"]
query_features = ["f_1_num_of_terms",  "f_2_query_length", "f_3_has_numbers", "f_4_has_special_char"]
y = df_filtered['neuralness']  # Target column

# Initialize cross-validation strategy
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

# Define the model
regr = Ridge(alpha=alpha)

rmse_scorer = make_scorer(root_mean_squared_error, greater_is_better=False)

# Initialize variables to track the best combination
best_score = float('inf')
best_combination = None
best_metrics = {}
df_model_metrics = []

# Try all feature combinations of length 2 to 9
for r in tqdm_notebook(range(2, len(feature_columns) + 1)):
    for combo in combinations(feature_columns, r):
        X = df_filtered[list(combo)]  # Select current feature subset
        
        # Perform cross-validation
        rmse_scores = cross_val_score(regr, X, y, cv=cv, scoring=rmse_scorer)
        
        # Calculate the mean metrics for the current combination
        mean_rmse = -np.mean(rmse_scores)  # Since RMSE scorer is negative for minimization

        # does the current combination contain keyword search features
        keyword_b = any(feature in combo for feature in kws_features)
        # does the current combination contain neural search features
        neural_b = any(feature in combo for feature in neural_search_features)
        # does the current combination contain only query features (no keyword or neural search features)
        query_b = any(feature in combo for feature in query_features)

        df_model_metrics.append(pd.DataFrame({
            "features": [combo]*5,
            "rmse_scores": rmse_scores,
            "keyword_f": [keyword_b]*5,
            "neural_f": [neural_b]*5,
            "query_f": [query_b]*5,
            "model": "lr",
            "regularization": True,
            }))
        
        # Update if the current combination is better
        if mean_rmse < best_score:
            best_score = mean_rmse
            best_combination = combo
            best_metrics = {
                'RMSE (mean)': mean_rmse
            }

df_metrics = pd.concat([df_metrics, pd.concat(df_model_metrics)])

# Output the best feature combination and its metrics
print("Best Feature Combination:", best_combination)
print("Best Metrics:")
print(f"Cross-Validation RMSE (mean): {best_metrics['RMSE (mean)']}")

  0%|          | 0/8 [00:00<?, ?it/s]

Best Feature Combination: ('f_1_num_of_terms', 'f_5_num_results', 'f_7_sum_title_scores', 'f_9_avg_semantic_score')
Best Metrics:
Cross-Validation RMSE (mean): 0.26376753118554336


## Random Forest Regression with Feature Combinations and Cross Validation

In [225]:
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, root_mean_squared_error, r2_score
from itertools import combinations
import numpy as np
import pandas as pd

# Prepare features and target
feature_columns = [ "f_1_num_of_terms", "f_2_query_length", "f_3_has_numbers", "f_4_has_special_char", "f_5_num_results",
                   "f_6_max_title_score", "f_7_sum_title_scores", "f_8_max_semantic_score",  "f_9_avg_semantic_score"]
kws_features = ["f_5_num_results", "f_6_max_title_score", "f_7_sum_title_scores"]
neural_search_features = ["f_8_max_semantic_score",  "f_9_avg_semantic_score"]
query_features = ["f_1_num_of_terms",  "f_2_query_length", "f_3_has_numbers", "f_4_has_special_char"]

y = df_filtered['neuralness']  # Target column

# Initialize cross-validation strategy
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

# Define the model
model = RandomForestRegressor(random_state=42)

rmse_scorer = make_scorer(root_mean_squared_error, greater_is_better=False)

# Initialize variables to track the best combination
best_score = float('inf')
best_combination = None
best_metrics = {}
df_model_metrics = []

# Try all feature combinations of length 2 to 9
for r in tqdm_notebook(range(2, len(feature_columns) + 1)):
    for combo in combinations(feature_columns, r):
        X = df_filtered[list(combo)]  # Select current feature subset
        
        # Perform cross-validation
        scores = cross_val_score(model, X, y, cv=cv)
        rmse_scores = cross_val_score(model, X, y, cv=cv, scoring=rmse_scorer)
        
        # Calculate the mean metrics for the current combination
        mean_score = scores.mean()
        std_score = scores.std()
        mean_rmse = -np.mean(rmse_scores)  # Since RMSE scorer is negative for minimization

        # does the current combination contain keyword search features
        keyword_b = any(feature in combo for feature in kws_features)
        # does the current combination contain neural search features
        neural_b = any(feature in combo for feature in neural_search_features)
        # does the current combination contain query features
        query_b = any(feature in combo for feature in query_features)
        
        df_model_metrics.append(pd.DataFrame({
            "features": [combo]*5,
            "rmse_scores": rmse_scores,
            "keyword_f": [keyword_b]*5,
            "neural_f": [neural_b]*5,
            "query_f": [query_b]*5,
            "model": "rf",
            "regularization": False,
            }))
        # Update if the current combination is better
        if mean_rmse < best_score:
            best_score = mean_rmse
            best_combination = combo
            best_metrics = {
                'RMSE (mean)': mean_rmse
            }

df_metrics = pd.concat([df_metrics, pd.concat(df_model_metrics)])

# Output the best feature combination and its metrics
print("Best Feature Combination:", best_combination)
print("Best Metrics:")
print(f"Cross-Validation RMSE (mean): {best_metrics['RMSE (mean)']}")

  0%|          | 0/8 [00:00<?, ?it/s]

Best Feature Combination: ('f_2_query_length', 'f_3_has_numbers', 'f_4_has_special_char', 'f_5_num_results', 'f_7_sum_title_scores')
Best Metrics:
Cross-Validation RMSE (mean): 0.27058291759236447


## Random Forest Regression with Feature Combinations, Cross Validation and Regularization (RandomizedSearchCV)

In [226]:
# RandomizedSearchCV is very verbose in logging warnings. The following snippet suppresses these.
# source: https://stackoverflow.com/questions/32612180/eliminating-warnings-from-scikit-learn
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [227]:
from sklearn.model_selection import ShuffleSplit, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, root_mean_squared_error, r2_score
from itertools import combinations
import numpy as np
import pandas as pd

# Prepare features and target
feature_columns = [ "f_1_num_of_terms", "f_2_query_length", "f_3_has_numbers", "f_4_has_special_char", "f_5_num_results",
                   "f_6_max_title_score", "f_7_sum_title_scores", "f_8_max_semantic_score",  "f_9_avg_semantic_score"]
kws_features = ["f_5_num_results", "f_6_max_title_score", "f_7_sum_title_scores"]
neural_search_features = ["f_8_max_semantic_score",  "f_9_avg_semantic_score"]
query_features = ["f_1_num_of_terms",  "f_2_query_length", "f_3_has_numbers", "f_4_has_special_char"]

y = df_filtered['neuralness']  # Target column

# Initialize cross-validation strategy
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

# Define Random Forest hyperparameter distributions for RandomizedSearchCV
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [1, 2, 3, 5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

rmse_scorer = make_scorer(root_mean_squared_error, greater_is_better=False)

# Initialize variables to track the best combination
best_score = float('inf')
best_combination = None
best_metrics = {}
df_model_metrics = []

# Try all feature combinations of length 2 to 9
for r in tqdm_notebook(range(2, len(feature_columns) + 1)):
    for combo in combinations(feature_columns, r):
        X = df_filtered[list(combo)]  # Select current feature subset

        # Define the model
        model = RandomForestRegressor(random_state=42)
        # Initialize RandomizedSearchCV
        random_search = RandomizedSearchCV(
            estimator=model,
            param_distributions=param_dist,
            n_iter=10,  # Number of random samples to draw
            scoring=rmse_scorer,
            cv=cv,
            random_state=42,
            n_jobs=-1,
            verbose=0
        )

        # Perform cross-validation with randomized hyperparameter search
        random_search.fit(X, y)

        # Get the best score and parameters from the random search
        mean_rmse = -random_search.best_score_  # Convert to positive RMSE
        best_params = random_search.best_params_

        # Retrieve mean_test_score and mask out NaNs
        mean_test_scores = -random_search.cv_results_["mean_test_score"]
        valid_scores = np.where(~np.isnan(mean_test_scores), mean_test_scores, np.inf)  # Replace NaNs with infinity
        
        # Find the index of the best configuration based on valid mean_test_scores
        best_config_index = np.argmin(valid_scores)
        
        # Retrieve individual fold scores for the best configuration
        best_fold_scores = []
        for fold_index in range(5):  # 5 folds
            # Get the score for the current fold of the best configuration
            score = random_search.cv_results_[f"split{fold_index}_test_score"][best_config_index]
            best_fold_scores.append(score)

        # does the current combination contain keyword search features
        keyword_b = any(feature in combo for feature in kws_features)
        # does the current combination contain neural search features
        neural_b = any(feature in combo for feature in neural_search_features)
        # does the current combination contain query features
        query_b = any(feature in combo for feature in query_features)

        df_model_metrics.append(pd.DataFrame({
            "features": [combo]*5,
            "rmse_scores": best_fold_scores,
            "keyword_f": [keyword_b]*5,
            "neural_f": [neural_b]*5,
            "query_f": [query_b]*5,
            "model": "rf",
            "regularization": True,
            }))
        
        # Update if the current combination is better
        if mean_rmse < best_score:
            best_score = mean_rmse
            best_combination = combo
            best_param_set = best_params
            best_metrics = {
                'RMSE (mean)': mean_rmse
            }

df_metrics = pd.concat([df_metrics, pd.concat(df_model_metrics)])

# Output the best feature combination and its metrics
print("Best Feature Combination:", best_combination)
print("Best Parameter Set:", best_param_set)
print("Best Metrics:")
print(f"Cross-Validation RMSE (mean): {best_metrics['RMSE (mean)']}")

  0%|          | 0/8 [00:00<?, ?it/s]

Best Feature Combination: ('f_4_has_special_char', 'f_5_num_results')
Best Parameter Set: {'n_estimators': 50, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'log2', 'max_depth': 3}
Best Metrics:
Cross-Validation RMSE (mean): 0.2560437740620821


### Aggregate Metrics per Model, Regularization, and Feature Combination

In [228]:
# multiply rmse scores by -1 as they are all negative for minimization
df_metrics['rmse_scores'] = df_metrics['rmse_scores'] * (-1)

In [229]:
# join features to a feature string
df_metrics['features'] =  df_metrics.features.apply(lambda x: ','.join(x))

In [230]:
df_agg_metrics = df_metrics.groupby(by=['features', 'keyword_f', 'neural_f', 'query_f', 'model', 'regularization']).agg({
    'rmse_scores': ['mean', 'var', np.std]
}).reset_index()
df_agg_metrics.columns = ['features', 'keyword_f', 'neural_f', 'query_f', 'model', 'regularization', 'mean_rmse', 'var_rmse', 'std_rmse']


In [231]:
df_agg_metrics.head(5)

,features,keyword_f,neural_f,query_f,model,regularization,mean_rmse,var_rmse,std_rmse
0,"f_1_num_of_terms,f_2_query_length",False,False,True,lr,False,0.268859,0.000808,0.028434
1,"f_1_num_of_terms,f_2_query_length",False,False,True,lr,True,0.268695,0.000803,0.028340
2,"f_1_num_of_terms,f_2_query_length",False,False,True,rf,False,0.292087,0.000029,0.005383
3,"f_1_num_of_terms,f_2_query_length",False,False,True,rf,True,0.273332,0.000441,0.021009
4,"f_1_num_of_terms,f_2_query_length,f_3_has_numbers",False,False,True,lr,False,0.271316,0.000823,0.028694


In [232]:
# 10 best feature combinations across all experiments
df_agg_metrics.sort_values(by='mean_rmse').head(10)

,features,keyword_f,neural_f,query_f,model,regularization,mean_rmse,var_rmse,std_rmse
1783,"f_4_has_special_char,f_5_num_results",True,False,True,rf,True,0.256044,0.000256,0.015999
1407,"f_2_query_length,f_5_num_results",True,False,True,rf,True,0.257924,0.000282,0.016787
1835,"f_4_has_special_char,f_5_num_results,f_8_max_s...",True,True,True,rf,True,0.258696,0.000429,0.020700
1283,"f_2_query_length,f_4_has_special_char,f_5_num_...",True,False,True,rf,True,0.258848,0.000324,0.017997
15,"f_1_num_of_terms,f_2_query_length,f_3_has_numb...",True,False,True,rf,True,0.259053,0.000301,0.017355
1335,"f_2_query_length,f_4_has_special_char,f_5_num_...",True,True,True,rf,True,0.259317,0.000304,0.017449
1031,"f_2_query_length,f_3_has_numbers,f_4_has_speci...",True,False,True,rf,True,0.259382,0.000325,0.018041
1907,"f_5_num_results,f_6_max_title_score",True,False,False,rf,True,0.259466,0.000380,0.019498
1787,"f_4_has_special_char,f_5_num_results,f_6_max_t...",True,False,True,rf,True,0.259555,0.000417,0.020417
1819,"f_4_has_special_char,f_5_num_results,f_7_sum_t...",True,False,True,rf,True,0.259560,0.000425,0.020604


In [233]:
# 10 worst feature combinations across all experiments
df_agg_metrics.sort_values(by='mean_rmse', ascending=False).head(10)

,features,keyword_f,neural_f,query_f,model,regularization,mean_rmse,var_rmse,std_rmse
1894,"f_4_has_special_char,f_8_max_semantic_score",False,True,True,rf,False,0.342811,0.000516,0.022710
1010,"f_1_num_of_terms,f_8_max_semantic_score",False,True,True,rf,False,0.335348,0.000469,0.021653
886,"f_1_num_of_terms,f_4_has_special_char,f_8_max_...",False,True,True,rf,False,0.331135,0.000379,0.019458
1878,"f_4_has_special_char,f_7_sum_title_scores",True,False,True,rf,False,0.330046,0.000487,0.022065
1770,"f_3_has_numbers,f_8_max_semantic_score",False,True,True,rf,False,0.330042,0.000503,0.022431
1646,"f_3_has_numbers,f_4_has_special_char,f_8_max_s...",False,True,True,rf,False,0.329870,0.000473,0.021760
1630,"f_3_has_numbers,f_4_has_special_char,f_7_sum_t...",True,False,True,rf,False,0.323936,0.000526,0.022926
1846,"f_4_has_special_char,f_6_max_title_score",True,False,True,rf,False,0.323361,0.000817,0.028581
1754,"f_3_has_numbers,f_7_sum_title_scores",True,False,True,rf,False,0.322077,0.000599,0.024474
758,"f_1_num_of_terms,f_3_has_numbers,f_8_max_seman...",False,True,True,rf,False,0.321825,0.000733,0.027066


### Visualization

In [234]:
pio.renderers.default = 'plotly_mimetype'
x = df_agg_metrics.sort_values(by='mean_rmse')['features'].head(10)

fig = px.box(df_metrics[df_metrics['features'].isin(x)], x='features',
             y='rmse_scores', title='RMSE Boxplot of Top 10 Feature Combinations')
fig.update_layout(height=800, width=1100,
                  title_text="Top 10 Feature Combinations",
                  xaxis_title="feature combinations", yaxis_title="rmse scores")
fig.update_xaxes(showticklabels=False)
fig.show()

#### Best Linear Models

In [235]:
x = df_agg_metrics[df_agg_metrics['model'] == 'lr'].sort_values(by='mean_rmse')['features'].head(10)

fig = px.box(df_metrics[(df_metrics['features'].isin(x)) & (df_metrics['model'] == 'lr')], x='features',
             y='rmse_scores', title='RMSE Boxplot of Top 10 Feature Combinations')
fig.update_layout(height=800, width=1100,
                  title_text="Top 10 Linear Regression Feature Combinations",
                  xaxis_title="feature combinations", yaxis_title="rmse scores")
fig.update_xaxes(showticklabels=False)
fig.show()

#### Best Random Forest Models

In [236]:
x = df_agg_metrics[df_agg_metrics['model'] == 'rf'].sort_values(by='mean_rmse')['features'].head(10)

fig = px.box(df_metrics[(df_metrics['features'].isin(x)) & (df_metrics['model'] == 'rf')], x='features',
             y='rmse_scores', title='RMSE Boxplot of Top 10 Feature Combinations')
fig.update_layout(height=800, width=1100,
                  title_text="Top 10 Random Forest Feature Combinations",
                  xaxis_title="feature combinations", yaxis_title="rmse scores")
fig.update_xaxes(showticklabels=False)
fig.show()

### Show the best feature combinations based on the mean root squared error grouped by the features they include

Groups: contains query features, contains keyword search features, and contains neural search features and best of all

In [237]:
df_agg_metrics['keyword_f_only'] = (df_agg_metrics['keyword_f'] == True) & (df_agg_metrics['neural_f'] == False) & (df_agg_metrics['query_f'] == False)
df_agg_metrics['neural_f_only'] = (df_agg_metrics['keyword_f'] == False) & (df_agg_metrics['neural_f'] == True) & (df_agg_metrics['query_f'] == False)
df_agg_metrics['query_f_only'] = (df_agg_metrics['keyword_f'] == False) & (df_agg_metrics['neural_f'] == False) & (df_agg_metrics['query_f'] == True)


In [238]:
df_agg_metrics.head(3)

,features,keyword_f,neural_f,query_f,model,regularization,mean_rmse,var_rmse,std_rmse,keyword_f_only,neural_f_only,query_f_only
0,"f_1_num_of_terms,f_2_query_length",False,False,True,lr,False,0.268859,0.000808,0.028434,False,False,True
1,"f_1_num_of_terms,f_2_query_length",False,False,True,lr,True,0.268695,0.000803,0.028340,False,False,True
2,"f_1_num_of_terms,f_2_query_length",False,False,True,rf,False,0.292087,0.000029,0.005383,False,False,True


In [239]:
# Create a better readable string to identify the feature combinations
# The string contains 'k' if it contains keyword search features, 'n' if contains neural search features and
# 'q' if it contains query string features

def create_feature_string(row):
    keyword_s = ""
    neural_s = ""
    query_s = ""
    if row.keyword_f == True:
        keyword_s = "k"
    if row.neural_f == True:
        neural_s = "n"
    if row.query_f == True:
        query_s = "q"
    return keyword_s + neural_s + query_s

In [240]:
# create a string as a key of the feature combination
df_metrics['f_str'] = df_metrics.apply(create_feature_string, axis=1)
df_metrics.head(3)

,features,rmse_scores,keyword_f,neural_f,query_f,model,regularization,f_str
0,"f_1_num_of_terms,f_2_query_length",0.240430,False,False,True,lr,False,q
1,"f_1_num_of_terms,f_2_query_length",0.246327,False,False,True,lr,False,q
2,"f_1_num_of_terms,f_2_query_length",0.290655,False,False,True,lr,False,q


### Linear Regression Model: Best Feature Combination Grouped by Feature types 

In [241]:
# Select the best three models that contain query features, neural features, keyword features and the best overall feature combinations
x_query_f = df_agg_metrics[(df_agg_metrics['query_f']==True) & (df_agg_metrics['model'] == 'lr')].sort_values(by='mean_rmse')['features'].head(5).values
x_neural_f = df_agg_metrics[(df_agg_metrics['neural_f']==True) & (df_agg_metrics['model'] == 'lr')].sort_values(by='mean_rmse')['features'].head(5).values
x_keyword_f = df_agg_metrics[(df_agg_metrics['keyword_f']==True) & (df_agg_metrics['model'] == 'lr')].sort_values(by='mean_rmse')['features'].head(5).values

x_all_f = df_agg_metrics[df_agg_metrics['model'] == 'lr'].sort_values(by='mean_rmse')['features'].head(5).values

x = np.concatenate([x_query_f, x_neural_f, x_keyword_f, x_all_f])

In [242]:
fig = px.box(df_metrics[(df_metrics['features'].isin(x)) & (df_metrics['model'] == 'lr')], x='features',
             y='rmse_scores', title='RMSE Boxplot of Best Feature Combinations', color='f_str')
fig.update_layout(height=800, width=1100,
                  title_text="Linear Model - Best Feature Combinations by Feature Types",
                  xaxis_title="feature combinations", yaxis_title="rmse scores")
fig.update_xaxes(showticklabels=False)
fig.show()

In [243]:
x_query_f = df_agg_metrics[(df_agg_metrics['query_f_only']==True) & (df_agg_metrics['model'] == 'lr')].sort_values(by='mean_rmse')['features'].head(3).values
x_neural_f = df_agg_metrics[(df_agg_metrics['neural_f_only']==True) & (df_agg_metrics['model'] == 'lr')].sort_values(by='mean_rmse')['features'].head(3).values
x_keyword_f = df_agg_metrics[(df_agg_metrics['keyword_f_only']==True) & (df_agg_metrics['model'] == 'lr')].sort_values(by='mean_rmse')['features'].head(3).values

x_all_f = df_agg_metrics[df_agg_metrics['model'] == 'lr'].sort_values(by='mean_rmse')['features'].head(3).values

x = np.concatenate([x_query_f, x_neural_f, x_keyword_f, x_all_f])

In [244]:
fig = px.box(df_metrics[(df_metrics['features'].isin(x)) & (df_metrics['model'] == 'lr')], x='features',
             y='rmse_scores', title='RMSE Boxplot of Best Feature Combinations', color='f_str')
fig.update_layout(height=800, width=1100,
                  title_text="Linear Model - Best Feature Combinations within Feature Groups",
                  xaxis_title="feature combinations", yaxis_title="rmse scores")
fig.update_xaxes(showticklabels=False)
fig.show()

### Random Forest Model: Best Feature Combination Grouped by Feature types 

In [245]:
# Select the best three models that contain query features, neural features, keyword features and the best overall feature combinations
x_query_f = df_agg_metrics[(df_agg_metrics['query_f']==True) & (df_agg_metrics['model'] == 'rf')].sort_values(by='mean_rmse')['features'].head(5).values
x_neural_f = df_agg_metrics[(df_agg_metrics['neural_f']==True) & (df_agg_metrics['model'] == 'rf')].sort_values(by='mean_rmse')['features'].head(5).values
x_keyword_f = df_agg_metrics[(df_agg_metrics['keyword_f']==True) & (df_agg_metrics['model'] == 'rf')].sort_values(by='mean_rmse')['features'].head(5).values

x_all_f = df_agg_metrics[df_agg_metrics['model'] == 'rf'].sort_values(by='mean_rmse')['features'].head(5).values

x = np.concatenate([x_query_f, x_neural_f, x_keyword_f, x_all_f])

In [246]:
fig = px.box(df_metrics[(df_metrics['features'].isin(x)) & (df_metrics['model'] == 'rf')], x='features',
             y='rmse_scores', title='RMSE Boxplot of Best Feature Combinations', color='f_str')
fig.update_layout(height=800, width=1100,
                  title_text="Random Forest - Best Feature Combinations by Feature Types",
                  xaxis_title="feature combinations", yaxis_title="rmse scores")
fig.update_xaxes(showticklabels=False)
fig.show()

In [247]:
x_query_f = df_agg_metrics[(df_agg_metrics['query_f_only']==True) & (df_agg_metrics['model'] == 'rf')].sort_values(by='mean_rmse')['features'].head(3).values
x_neural_f = df_agg_metrics[(df_agg_metrics['neural_f_only']==True) & (df_agg_metrics['model'] == 'rf')].sort_values(by='mean_rmse')['features'].head(3).values
x_keyword_f = df_agg_metrics[(df_agg_metrics['keyword_f_only']==True) & (df_agg_metrics['model'] == 'rf')].sort_values(by='mean_rmse')['features'].head(3).values

x_all_f = df_agg_metrics[df_agg_metrics['model'] == 'rf'].sort_values(by='mean_rmse')['features'].head(3).values

x = np.concatenate([x_query_f, x_neural_f, x_keyword_f, x_all_f])

In [248]:
fig = px.box(df_metrics[(df_metrics['features'].isin(x)) & (df_metrics['model'] == 'rf')], x='features',
             y='rmse_scores', title='RMSE Boxplot of Best Feature Combinations', color='f_str')
fig.update_layout(height=800, width=1100,
                  title_text="Random Forest - Best Feature Combinations within Feature Groups",
                  xaxis_title="feature combinations", yaxis_title="rmse scores")
fig.update_xaxes(showticklabels=False)
fig.show()

In [249]:
fig = make_subplots(rows=1, cols=4, shared_yaxes=True)

fig.add_trace(go.Box(y=df_agg_metrics[df_agg_metrics['regularization'] == False].sort_values(by=['mean_rmse']).head(10)['mean_rmse'],
                     name="all combinations"),
              row=1, col=1)

fig.add_trace(go.Box(y=df_agg_metrics[(df_agg_metrics['keyword_f_only'] == True) & (df_agg_metrics['regularization'] == False)].head(10)['mean_rmse'],
                    name="only keyword features"),
              row=1, col=2)

fig.add_trace(go.Box(y=df_agg_metrics[(df_agg_metrics['neural_f_only'] == True) & (df_agg_metrics['regularization'] == False)].sort_values(by=['mean_rmse']).head(10)['mean_rmse'],
                    name="only neural features"),
              row=1, col=3)

fig.add_trace(go.Box(y=df_agg_metrics[(df_agg_metrics['query_f_only'] == True) & (df_agg_metrics['regularization'] == False)].sort_values(by=['mean_rmse']).head(10)['mean_rmse'],
                    name="only query features"),
              row=1, col=4)

fig.update_layout(height=800, width=1100,
                  title_text="Top 10 Feature Combinations within Feature Groups")
fig.show()

In [250]:
fig = make_subplots(rows=1, cols=4, shared_yaxes=True)

fig.add_trace(go.Box(y=df_agg_metrics[(df_agg_metrics['model'] == 'lr') & (df_agg_metrics['regularization'] == False)].sort_values(by=['mean_rmse']).head(10)['mean_rmse'],
                     name="all combinations"),
              row=1, col=1)

fig.add_trace(go.Box(y=df_agg_metrics[(df_agg_metrics['keyword_f_only'] == True) & (df_agg_metrics['model'] == 'lr') & (df_agg_metrics['regularization'] == False)].head(10)['mean_rmse'],
                    name="only keyword features"),
              row=1, col=2)

fig.add_trace(go.Box(y=df_agg_metrics[(df_agg_metrics['neural_f_only'] == True) & (df_agg_metrics['model'] == 'lr') & (df_agg_metrics['regularization'] == False)].sort_values(by=['mean_rmse']).head(10)['mean_rmse'],
                    name="only neural features"),
              row=1, col=3)

fig.add_trace(go.Box(y=df_agg_metrics[(df_agg_metrics['query_f_only'] == True) & (df_agg_metrics['model'] == 'lr') & (df_agg_metrics['regularization'] == False)].sort_values(by=['mean_rmse']).head(10)['mean_rmse'],
                    name="only query features"),
              row=1, col=4)

fig.update_layout(height=800, width=1100,
                  title_text="Linear Model - Top 10 Feature Combinations within Feature Groups")
fig.show()

In [251]:
fig = make_subplots(rows=1, cols=4, shared_yaxes=True)

fig.add_trace(go.Box(y=df_agg_metrics[(df_agg_metrics['model'] == 'rf') & (df_agg_metrics['regularization'] == False)].sort_values(by=['mean_rmse']).head(10)['mean_rmse'],
                     name="all combinations"),
              row=1, col=1)

fig.add_trace(go.Box(y=df_agg_metrics[(df_agg_metrics['keyword_f_only'] == True) & (df_agg_metrics['model'] == 'rf') & (df_agg_metrics['regularization'] == False)].head(10)['mean_rmse'],
                    name="only keyword features"),
              row=1, col=2)

fig.add_trace(go.Box(y=df_agg_metrics[(df_agg_metrics['neural_f_only'] == True) & (df_agg_metrics['model'] == 'rf') & (df_agg_metrics['regularization'] == False)].sort_values(by=['mean_rmse']).head(10)['mean_rmse'],
                    name="only neural features"),
              row=1, col=3)

fig.add_trace(go.Box(y=df_agg_metrics[(df_agg_metrics['query_f_only'] == True) & (df_agg_metrics['model'] == 'rf') & (df_agg_metrics['regularization'] == False)].sort_values(by=['mean_rmse']).head(10)['mean_rmse'],
                    name="only query features"),
              row=1, col=4)

fig.update_layout(height=800, width=1100,
                  title_text="Random Forest - Top 10 Feature Combinations within Feature Groups")
fig.show()

# Train Models

With the above exploration and analysis done, we can train a linear regression model and a random forest regressor by using some of the insights the analysis produced.

## Train a Linear Regression Model with cross-validation

In [270]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import ShuffleSplit, cross_val_score, train_test_split

# Prepare features and target - the best feature combination based on the previous exploration
#feature_columns = [ "f_1_num_of_terms", "f_2_query_length", "f_3_has_numbers", "f_4_has_special_char", "f_5_num_results",
#                   "f_6_max_title_score", "f_7_sum_title_scores", "f_8_max_semantic_score",  "f_9_avg_semantic_score"]
feature_columns = ['f_1_num_of_terms', 'f_5_num_results', 'f_7_sum_title_scores', 'f_9_avg_semantic_score']
X = df_filtered[feature_columns]   # Select feature columns
y = df_filtered['neuralness']      # Select the target column

# Initialize cross-validation strategy
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

# Define the model - choose the best option based on experiments above
#regr = LinearRegression()
regr = Ridge(alpha=alpha)

# Perform cross-validation
rmse_scorer = make_scorer(root_mean_squared_error)

rmse_scores = cross_val_score(regr, X, y, cv=cv, scoring=rmse_scorer)

# Output the results
print("Cross-Validation Root Mean Squared Error:", np.mean(rmse_scores))

# Split data into training and testing sets
# Do that manually to have the identical distribution as with the baseline and hybrid search optimizer runs 
# Otherwise no fair comparison
X_train = df_filtered[df_filtered['query_string'].isin(df_metrics_train['query'])][feature_columns]
y_train = df_filtered[df_filtered['query_string'].isin(df_metrics_train['query'])]['neuralness']
X_test = df_filtered[df_filtered['query_string'].isin(df_metrics_test['query'])][feature_columns]
y_test = df_filtered[df_filtered['query_string'].isin(df_metrics_test['query'])]['neuralness']

regr.fit(X_train, y_train)

y_regr_pred = regr.predict(X_test)

# Evaluate the model
rmse = root_mean_squared_error(y_test, y_regr_pred)
r2 = r2_score(y_test, y_regr_pred)

print("Root Mean Squared Error:", rmse)
print("R^2 Score:", r2)

Cross-Validation Root Mean Squared Error: 0.26376753118554336
Root Mean Squared Error: 0.18903543246643553
R^2 Score: -2.899118523228995e+30


## Train a Random Forest Regression Model with cross-validation

The RandomForestRegressor parameters were the best set identified as part of a series of experiments. See https://github.com/o19s/opensearch-hybrid-search-optimization/issues/1 for additional information.

In [271]:
### Manually define the train/test split

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, root_mean_squared_error, r2_score
import numpy as np

# Prepare features and target
#feature_columns = [ "f_1_num_of_terms", "f_2_query_length", "f_3_has_numbers", "f_4_has_special_char", "f_5_num_results",
#                   "f_6_max_title_score", "f_7_sum_title_scores", "f_8_max_semantic_score",  "f_9_avg_semantic_score"]
feature_columns = ['f_4_has_special_char', 'f_5_num_results']
X = df_filtered[feature_columns]   # Select feature columns
y = df_filtered['neuralness']      # Select the target column

# Initialize cross-validation strategy
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

# Define the model - based on experiment results above
model = RandomForestRegressor(n_estimators=50, min_samples_split=2, min_samples_leaf=4, max_features="log2", 
                                max_depth=3, random_state=42)

# Perform cross-validation
rmse_scorer = make_scorer(root_mean_squared_error)

rmse_scores = cross_val_score(model, X, y, cv=cv, scoring=rmse_scorer)

# Output the results
print("Cross-Validation Root Mean Squared Error:", np.mean(rmse_scores))

# Split data into training and testing sets
# Do that manually to have the identical distribution as with the baseline and hybrid search optimizer runs 
# Otherwise no fair comparison
X_train = df_filtered[df_filtered['query_string'].isin(df_metrics_train['query'])][feature_columns]
y_train = df_filtered[df_filtered['query_string'].isin(df_metrics_train['query'])]['neuralness']
X_test = df_filtered[df_filtered['query_string'].isin(df_metrics_test['query'])][feature_columns]
y_test = df_filtered[df_filtered['query_string'].isin(df_metrics_test['query'])]['neuralness']

# Train the model on the training data
model.fit(X_train, y_train)

# Predict on the test data
y_pred = model.predict(X_test)

# Evaluate the model
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Root Mean Squared Error:", rmse)
print("R^2 Score:", r2)

Cross-Validation Root Mean Squared Error: 0.2560437740620821
Root Mean Squared Error: 0.20623408477564495
R^2 Score: -3.450646161157185e+30


In [272]:
# Store the test data for later usage
df_filtered.loc[df_filtered.index.isin(X_test.index)].to_csv('../data/dynamic_optimizer_test_data.csv', index=False)

## Save Models

In [273]:
import pickle

with open('regression_model.pkl', 'wb') as fid:
    pickle.dump(regr, fid) 

with open('random_forest_model.pkl', 'wb') as fid:
    pickle.dump(model, fid) 